In [1]:
from flask import Flask, render_template, request
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import os
import pandas as pd
import numpy as np

In [15]:
driver = webdriver.Chrome(keep_alive=True)
driver.set_window_size(1100, 800)
print("Clicker starts")
username = "info@waddystore.com"
password = "Waddy1208"
driver.get('https://admin.shoplineapp.com/admin/waddystore/')
driver.find_element(By.ID, "staff_email").send_keys(username)
driver.find_element(By.ID, "staff_password").send_keys(password)
driver.find_element(By.ID, "reg-submit-button").click()
print("Login sucessful")

C:\Users\Waddy Store\AppData\Local\Temp\ipykernel_33040\110291425.py:1: DeprecationWarning: keep_alive has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(keep_alive=True)


Clicker starts
Login sucessful


In [16]:
#check path
print(os.path.abspath(os.getcwd()))

c:\Users\Waddy Store\Documents\Code\codespaces-flask\Update


In [17]:
os.chdir("..")
print(os.path.abspath(os.getcwd()))

c:\Users\Waddy Store\Documents\Code\codespaces-flask


In [18]:
# cross check with stock level
os.chdir("Check")
file_path = os.listdir()[0]
stock_df = pd.read_excel(file_path,index_col=None, header=0)
final_list = []
os.chdir("..")
os.chdir("Update")
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".xls"):
        file_path = f"{file}" 
        df, havestock_df, preorder_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        
        df = pd.read_excel(file_path,index_col=None, header=0)
        df = df.drop(index=0)
        preorder_df = df[df['Preorder Product'] == "Y"]
        havestock_df = preorder_df[preorder_df['Quantity (DO NOT EDIT)'] > 0]
        for barcode in havestock_df["Barcode"]:
            check_stock_df = stock_df.loc[stock_df['商品條碼'] == barcode]
            if int(check_stock_df['預設倉庫']) > 0:
                tmp_df = pd.DataFrame()
                tmp_df = df.loc[df['Barcode'] == barcode]
                if tmp_df.shape[0] > 0:
                    for sku_id in tmp_df['Product ID (DO NOT EDIT)']:
                        print("Now process: " , tmp_df['Product Name (Traditional Chinese)'], tmp_df['Barcode'])
                        driver.get("https://admin.shoplineapp.com/admin/waddystore/products/"+sku_id+"/edit")
                        print("Now browsing to SKU: " + sku_id)
                        varient_qty = tmp_df["Variant Quantity (DO NOT EDIT)"]
                        print(np.isnan(varient_qty.tolist()[0]))
                        if np.isnan(varient_qty.tolist()[0]):
                            driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[4]/a').click()
                            print("Go to Price and Quantity Tab")
                            accept_button = driver.find_element(By.XPATH,'//*[@id="productForm-pricing"]/div/div[3]/div[2]/div[1]/div/div[2]/div/div[2]/label/input')

                            if accept_button.is_selected:
                                accept_button.click()
                                print("Unticked 缺貨時可接單")
                            elif not accept_button.is_selected:
                                print("Already Unticked 缺貨時可接單")

                            
                        else:
                            driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[5]/a').click()
                            print("Go to Variations Tab")
                            accept_button = driver.find_element(By.XPATH,'//*[@id="productForm-variations"]/div/div[3]/div[3]/div[1]/div/div/div[2]/div/div[5]/label/input')
                            if accept_button.is_selected:
                                accept_button.click()
                                print("Unticked 缺貨時可接單")
                            elif not accept_button.is_selected:
                                print("Already Unticked 缺貨時可接單")

                        driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[8]/a').click()
                        print("Go to Settings Tab")

                        pre_order_switch = driver.find_element(By.XPATH,'//*[@id="productForm-settings"]/div[1]/div[3]/div[1]/div/div[2]/div/div[1]/div')
                        pre_order_switch_classess = pre_order_switch.get_attribute("class")
                        if "switch-on" in pre_order_switch_classess:
                            driver.find_element(By.XPATH,'//*[@id="productForm-settings"]/div[1]/div[3]/div[1]/div/div[2]/div/div[1]/div/span[2]').click()
                            print("Switched off Preorder Product Setting")
                        else:
                            print("Skip 2")

                        driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[1]/div/span[2]/button/span[2]').click()
                        print("Saved changes, completed")

                        final_list.append(tmp_df['Product Name (Traditional Chinese)'].tolist()[0])
                        print("Complete SKU")

print(final_list)

Now process:  269    SVAKOM Vick Neo 後庭按摩器 App遙控版
Name: Product Name (Traditional Chinese), dtype: object 269    6959633152355
Name: Barcode, dtype: object
Now browsing to SKU: 5f40bc6a754071003c197c05
True
Go to Price and Quantity Tab


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x00A76643]
	(No symbol) [0x00A0BE21]
	(No symbol) [0x0090D960]
	(No symbol) [0x009423A7]
	(No symbol) [0x00936B13]
	(No symbol) [0x0095FD7C]
	(No symbol) [0x0093641F]
	(No symbol) [0x009600D4]
	(No symbol) [0x00976B09]
	(No symbol) [0x0095FB76]
	(No symbol) [0x009349C1]
	(No symbol) [0x00935E5D]
	GetHandleVerifier [0x00CEA142+2497106]
	GetHandleVerifier [0x00D185D3+2686691]
	GetHandleVerifier [0x00D1BB9C+2700460]
	GetHandleVerifier [0x00B23B10+635936]
	(No symbol) [0x00A14A1F]
	(No symbol) [0x00A1A418]
	(No symbol) [0x00A1A505]
	(No symbol) [0x00A2508B]
	BaseThreadInitThunk [0x759E6BD9+25]
	RtlGetFullPathName_UEx [0x77058FD2+1218]
	RtlGetFullPathName_UEx [0x77058F9D+1165]
